# Import Libraries

In [16]:
import gurobipy as grb
import pandas as pd
from gurobipy import GRB
import numpy as np

# Model

In [17]:
 # Create a model
model = grb.Model("Model")

# Get data from json

In [18]:
# get the  data from the json file small.json
import json
with open('small.json') as f:
    data = json.load(f)

## Constants from data

In [19]:
horizon = data['horizon']
qualifications = data['qualifications']
staff_names = [i['name'] for i in data['staff']]
staff_qualifications = [i['qualifications'] for i in data['staff']]
job_list = [i['name'] for i in data['jobs']]
jour_list = [i for i in range(1,horizon+1)]
job_penalty = [i['daily_penalty'] for i in data['jobs']] 
job_qualifications = [i['working_days_per_qualification'] for i in data['jobs']]
job_gain = [i['gain'] for i in data['jobs']]
due_dates = [i['due_date'] for i in data['jobs']]
vacation_staff = [i['vacations'] for i in data['staff']]

# Model's parameters

In [20]:
# create a function that converts a dictionary to a matrix
def get_matrix(liste):
    test=pd.DataFrame.from_dict(liste,orient='index')
    test.fillna(0,inplace=True)
    return test.to_dict(orient='index')

In [21]:
# create dictionaries with job_list as key and job_penality, job_gain ans due_dates as values
job_penalty_dict = dict(zip(job_list,job_penalty))
job_gain_dict = dict(zip(job_list,job_gain))
due_dates_dict = dict(zip(job_list,due_dates))

# create a dictionary for staff qualifications
pi_c = {}
for i in range(len(staff_names)) :
    k = staff_names[i]
    pi_c[k] = {}
    for c in qualifications:
        if c in data['staff'][i]['qualifications']:
            pi_c[k][c] = 1
        else:
            pi_c[k][c] = 0

qk_c = get_matrix(dict(zip(job_list, job_qualifications)))

# get pi_j : 0 if vacation
staff_in_vacation = pd.DataFrame(index=staff_names,columns=jour_list)
# fill the dataframe staff_in_vacation with the values of the dictionary
for i in staff_names:
    for j in jour_list:
        staff_in_vacation.loc[i,j]=0 if j in dict(zip(staff_names,vacation_staff))[i] else 1
pi_j=staff_in_vacation.to_dict(orient='index')

# Sets

In [22]:
H = jour_list # t
Q = qualifications # k
S = staff_names # i
J = job_list # j

Qi, Qj, Vi = {}, {}, {}

# qualifications of staff i
for i in S:
    Qi[i] = [c for c in qualifications if pi_c[i][c] == 1]
# vacation of staff i
for i in S:
    Vi[i] = [j for j in jour_list if pi_j[i][j] == 0]
# qualifications of job j
for j in J:
    Qj[j] = [c for c in qualifications if qk_c[j][c] == 1]
# work load for job j with qualification k
# qk_c[j][k]

# gains
Gj = [job_gain_dict[j] for j in J]
# penalties
Pj = [job_penalty_dict[j] for j in J]
# due dates
Dj = [due_dates_dict[j] for j in J]

# Decision variables

In [23]:
X_i_j_k_t = model.addVars(staff_names, job_list, qualifications, jour_list, vtype=GRB.BINARY, name="X_i_j_k_t")

Y_j = model.addVars(job_list, vtype=GRB.BINARY, name="Y_j")

L_j = model.addVars(job_list, vtype=GRB.INTEGER, name="L_j")

E_j = model.addVars(job_list, vtype=GRB.INTEGER, name="E_j")

# Constraints

In [24]:
constr1={f'constr1{i}_{t}':model.addConstr( grb.quicksum(X_i_j_k_t for j in J for k in Q) <= 1 , name=f"constr1{i}_{t}")
    for i in S
    for t in H}

constr2={f'constr2{i}_{t}':model.addConstr( grb.quicksum(X_i_j_k_t for j in J for k in Q) <= 0 , name=f"constr2{i}_{t}")
    for i in S
    for t in Vi[i]}

# Xijkt =0 if wrong qualifications
constr3={f'constr3{i}_{j}_{t}':model.addConstr( grb.quicksum(X_i_j_k_t for k in Q if k not in Qj[j]) <= 0 , name=f"constr3{i}_{j}_{t}")
    for i in S
    for j in J
    for t in H}

KeyError: ('Olivia', 'Job1', 1, 'A')

In [ ]:
contr12={f'contr12{j}':model.addConstr( E_j[k] >= X_i_j_k_t[i,j,k,t]*t , name=f"contr12{j}")
    for i in S
    for t in H 
    for j in J
    for k in Q}

contr13={f'contr13{j}':model.addConstr( E_j[j] - Dj[j] <= L_j[j] , name=f"contr13{j}")
    for j in J}

In [ ]:
# Fonction Objectif
model.setObjective(grb.quicksum((Gj[j]*Y_j[j]) - L_j[j]*Pj[j] for j in J) , GRB.MAXIMIZE)

# Paramétrage (mode mute)
# model.params.outputflag = 0
# Résolution du PL
model.optimize()

In [ ]:
#model.computeIIS()
model.write("model.lp")
#  print constraints of the gurobipy.Model

In [ ]:
model.ObjVal

80.0

In [ ]:
values= []
for k, v in X_i_j_k_t.items():
    values.append(v.x)

In [ ]:
res = pd.DataFrame(list(X_i_j_k_t.keys()))
res[4] = values
result = res[res[4] == 1]
result

,0,1,2,3,4


## Result Table

In [ ]:
def color_table(x):
    if pd.isna(x):
        return "background-color: white"
    else:
        if "Job1" in x :
            return "background-color: blue"
        elif "Job2" in x:
            return "background-color: black"
        elif "Job3" in x:
            return "background-color: orange"
        elif "Job4" in x:
           return "background-color: grey"
        elif "Job5" in x:
           return "background-color: green"
        else:
           return  "background-color: red"

In [ ]:
df = pd.DataFrame(columns = [i for i in range(1,horizon+1)], index = staff_names)

for ind, val in result.iterrows():
    col = val[2]
    row = val[0]
    v = val[3] + " " + val[1]
    df.loc[row,col] = v
    # vacation
    for staff in staff_names:
        for day in jour_list:
            if pi_j[staff][day] == 0:
                df.loc[staff,day] = 'X'

df.style.applymap(color_table)

,1,2,3,4,5
Olivia,nan,nan,nan,nan,nan
Liam,nan,nan,nan,nan,nan
Emma,nan,nan,nan,nan,nan


In [ ]:
end_k

{'Job1': <gurobi.Var end_k[Job1] (value -0.0)>,
 'Job2': <gurobi.Var end_k[Job2] (value -0.0)>,
 'Job3': <gurobi.Var end_k[Job3] (value -0.0)>,
 'Job4': <gurobi.Var end_k[Job4] (value -0.0)>,
 'Job5': <gurobi.Var end_k[Job5] (value -0.0)>}

In [ ]:
delay_k

{'Job1': <gurobi.Var delay_k[Job1] (value -0.0)>,
 'Job2': <gurobi.Var delay_k[Job2] (value -0.0)>,
 'Job3': <gurobi.Var delay_k[Job3] (value -0.0)>,
 'Job4': <gurobi.Var delay_k[Job4] (value -0.0)>,
 'Job5': <gurobi.Var delay_k[Job5] (value -0.0)>}

In [ ]:
zk_j

{('Job1', 1): <gurobi.Var zk_j[Job1,1] (value -0.0)>,
 ('Job1', 2): <gurobi.Var zk_j[Job1,2] (value -0.0)>,
 ('Job1', 3): <gurobi.Var zk_j[Job1,3] (value -0.0)>,
 ('Job1', 4): <gurobi.Var zk_j[Job1,4] (value -0.0)>,
 ('Job1', 5): <gurobi.Var zk_j[Job1,5] (value -0.0)>,
 ('Job2', 1): <gurobi.Var zk_j[Job2,1] (value -0.0)>,
 ('Job2', 2): <gurobi.Var zk_j[Job2,2] (value -0.0)>,
 ('Job2', 3): <gurobi.Var zk_j[Job2,3] (value -0.0)>,
 ('Job2', 4): <gurobi.Var zk_j[Job2,4] (value -0.0)>,
 ('Job2', 5): <gurobi.Var zk_j[Job2,5] (value -0.0)>,
 ('Job3', 1): <gurobi.Var zk_j[Job3,1] (value -0.0)>,
 ('Job3', 2): <gurobi.Var zk_j[Job3,2] (value -0.0)>,
 ('Job3', 3): <gurobi.Var zk_j[Job3,3] (value -0.0)>,
 ('Job3', 4): <gurobi.Var zk_j[Job3,4] (value -0.0)>,
 ('Job3', 5): <gurobi.Var zk_j[Job3,5] (value -0.0)>,
 ('Job4', 1): <gurobi.Var zk_j[Job4,1] (value -0.0)>,
 ('Job4', 2): <gurobi.Var zk_j[Job4,2] (value -0.0)>,
 ('Job4', 3): <gurobi.Var zk_j[Job4,3] (value -0.0)>,
 ('Job4', 4): <gurobi.Var zk

In [ ]:
# for k in job_list:
for j in jour_list:
    print(zk_j['Job5',j])

<gurobi.Var zk_j[Job5,1] (value -0.0)>
<gurobi.Var zk_j[Job5,2] (value -0.0)>
<gurobi.Var zk_j[Job5,3] (value -0.0)>
<gurobi.Var zk_j[Job5,4] (value -0.0)>
<gurobi.Var zk_j[Job5,5] (value -0.0)>


In [ ]:
for j in jour_list:
    for s in staff_names:
        for c in  qualifications:
            print(X_i_j_k_t[s,'Job1',j,c])

<gurobi.Var pi_k_j_c[Olivia,Job1,1,A] (value -0.0)>
<gurobi.Var pi_k_j_c[Olivia,Job1,1,B] (value -0.0)>
<gurobi.Var pi_k_j_c[Olivia,Job1,1,C] (value -0.0)>
<gurobi.Var pi_k_j_c[Liam,Job1,1,A] (value -0.0)>
<gurobi.Var pi_k_j_c[Liam,Job1,1,B] (value -0.0)>
<gurobi.Var pi_k_j_c[Liam,Job1,1,C] (value -0.0)>
<gurobi.Var pi_k_j_c[Emma,Job1,1,A] (value -0.0)>
<gurobi.Var pi_k_j_c[Emma,Job1,1,B] (value -0.0)>
<gurobi.Var pi_k_j_c[Emma,Job1,1,C] (value -0.0)>
<gurobi.Var pi_k_j_c[Olivia,Job1,2,A] (value -0.0)>
<gurobi.Var pi_k_j_c[Olivia,Job1,2,B] (value -0.0)>
<gurobi.Var pi_k_j_c[Olivia,Job1,2,C] (value -0.0)>
<gurobi.Var pi_k_j_c[Liam,Job1,2,A] (value -0.0)>
<gurobi.Var pi_k_j_c[Liam,Job1,2,B] (value -0.0)>
<gurobi.Var pi_k_j_c[Liam,Job1,2,C] (value -0.0)>
<gurobi.Var pi_k_j_c[Emma,Job1,2,A] (value -0.0)>
<gurobi.Var pi_k_j_c[Emma,Job1,2,B] (value -0.0)>
<gurobi.Var pi_k_j_c[Emma,Job1,2,C] (value -0.0)>
<gurobi.Var pi_k_j_c[Olivia,Job1,3,A] (value -0.0)>
<gurobi.Var pi_k_j_c[Olivia,Job1,3,B